# Applying the Expected Context Framework to the Switchboard Corpus

### Using `DualContextWrapper`

This notebook demonstrates how our implementation of the Expected Context Framework can be applied to the Switchboard dataset. See [this dissertation](https://tisjune.github.io/research/dissertation) for more details about the framework, and more comments on the below analyses.

This notebook will show how to apply `DualContextWrapper`, a wrapper transformer that keeps track of two instances of `ExpectedContextModelTransformer`. For a version of this demo that initializes two separate instances of `ExpectedContextModelTransformer` instead, and that more explicitly demonstrates that functionality, see [this notebook](https://github.com/CornellNLP/ConvoKit/blob/ecf/convokit/expected_context_framework/demos/switchboard_exploration_demo.ipynb).



In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import math
import os

## 1. Loading and preprocessing the dataset

For this demo, we'll use the Switchboard corpus---a collection of telephone conversations which have been annotated with various dialog acts. More information on the dataset, as it exists in ConvoKit format, can be found [here](https://convokit.cornell.edu/documentation/switchboard.html); the original data is described [here](https://web.stanford.edu/~jurafsky/ws97/CL-dialog.pdf).

We will actually use a preprocessed version of the Switchboard corpus, which we can access below. Since Switchboard consists of transcribed telephone conversations, there are many disfluencies and backchannels, that make utterances messier, and that make it hard to identify what counts as an actual turn. In the version of the corpus we consider, for the purpose of demonstration, we remove the disfluencies and backchannels (acknowledging that we're discarding important parts of the conversations). 

In [3]:
from convokit import Corpus
from convokit import download

In [ ]:
# OPTION 1: DOWNLOAD CORPUS 
# UNCOMMENT THESE LINES TO DOWNLOAD CORPUS
# DATA_DIR = '<YOUR DIRECTORY>'
# SW_CORPUS_PATH = download('switchboard-processed-corpus', data_dir=DATA_DIR)

# OPTION 2: READ PREVIOUSLY-DOWNLOADED CORPUS FROM DISK
# UNCOMMENT THIS LINE AND REPLACE WITH THE DIRECTORY WHERE THE TENNIS-CORPUS IS LOCATED
# SW_CORPUS_PATH = '<YOUR DIRECTORY>'

In [5]:
sw_corpus = Corpus(SW_CORPUS_PATH)

In [6]:
sw_corpus.print_summary_stats()

Number of Speakers: 440
Number of Utterances: 44402
Number of Conversations: 1155


In [7]:
utt_eg_id = '3496-79'

as input, we use a preprocessed version of the utterance that only contains alphabetical words, found in the `alpha_text` metadata field.

In [8]:
sw_corpus.get_utterance(utt_eg_id).meta['alpha_text']

'How old were you when you left'

In order to avoid capturing topic-specific information, we restrict our analyses to a vocabulary of unigrams that occurs across many topics, and across many conversations:

In [9]:
from collections import defaultdict

In [10]:
topic_counts = defaultdict(set)
for ut in sw_corpus.iter_utterances():
    topic = sw_corpus.get_conversation(ut.conversation_id).meta['topic']
    for x in set(ut.meta['alpha_text'].lower().split()):
        topic_counts[x].add(topic)
topic_counts = {x: len(y) for x, y in topic_counts.items()}

word_convo_counts = defaultdict(set)
for ut in sw_corpus.iter_utterances():
    for x in set(ut.meta['alpha_text'].lower().split()):
        word_convo_counts[x].add(ut.conversation_id)
word_convo_counts = {x:  len(y) for x, y in word_convo_counts.items()}

min_topic_words = set(x for x,y in topic_counts.items() if y >= 33)
min_convo_words = set(x for x,y in word_convo_counts.items() if y >= 200)
vocab = sorted(min_topic_words.intersection(min_convo_words))

In [11]:
len(vocab)

381

In [12]:
from convokit.expected_context_framework import ColNormedTfidfTransformer, DualContextWrapper

## 2. Applying the Expected Context Framework

To apply the Expected Context Framework, we start by converting the input utterance text to an input vector representation. Here, we represent utterances in a term-document matrix that's _normalized by columns_ (empirically, we found that this ensures that the representations derived by the framework aren't skewed by the relative frequency of utterances). We use `ColNormedTfidfTransformer` transformer to do this:

In [13]:
tfidf_obj = ColNormedTfidfTransformer(input_field='alpha_text', output_field='col_normed_tfidf', binary=True, vocabulary=vocab)
_ = tfidf_obj.fit(sw_corpus)
_ = tfidf_obj.transform(sw_corpus)

We now use the Expected Context Framework. In short, the framework derives vector representations, and other characterizations, of terms and utterances that are based on their _expected conversational context_---i.e., the replies we expect will follow a term or utterance, or the preceding utterances that we expect the term/utterance will reply to. 

We are going to derive characterizations based both on the _forwards_ context, i.e., the expected replies, and the _backwards_ context, i.e., the expected predecessors. We'll apply the framework in each direction, and then compare the characterizations that result. To take care of both interlocked models, we use the `DualContextWrapper` transformer, which will keep track of two `ExpectedContextModelTransformer`s: one that relates utterances to predecessors (`reply_to`), and that outputs utterance-level attributes with the prefix `bk`; the other that relates utterances to replies (`next_id`) and outputs utterance-level attributes with the prefix `fw`. These parameters are specified via the `context_fields` and `output_prefixes` arguments.

Other arguments passed:
* `vect_field` and `context_vect_field` respectively denote the input vector representations of utterances and context utterances that `ec_fw` will work with. Here, we'll use the same tf-idf representations that we just computed above.
* `n_svd_dims` denotes the dimensionality of the vector representations that `ec_fw` will output. This is something that you can play around with---for this dataset, we found that more dimensions resulted in messier output, and a coarser, lower-dimensional representation was slightly more interpretable. (Technical note: technically, `ec_fw` produces vector representations of dimension `n_svd_dims`-1, since by default, it removes the first latent dimension, which we find tends to strongly reflect term frequency.)
* `n_clusters` denotes the number of utterance types that `ec_fw` will infer, given the representations it computes. Note that this is an interpretative step: looking at clusters of utterances helps us get a sense of what information the representations are capturing; this value does not actually impact the representations and other characterizations we derive.
* `random_state` and `cluster_random_state` are fixed for this demo, so we produce deterministic output.

In [14]:
dual_context_model = DualContextWrapper(context_fields=['reply_to','next_id'], output_prefixes=['bk','fw'],
                                    vect_field='col_normed_tfidf', context_vect_field='col_normed_tfidf', 
                                      n_svd_dims=15, n_clusters=2,
                                     random_state=1000, cluster_random_state=1000)

We'll fit the transformer on the subset of utterances and replies that have at least 5 unigrams from our vocabulary.

In [15]:
dual_context_model.fit(sw_corpus,selector=lambda x: x.meta.get('col_normed_tfidf__n_feats',0)>=5, 
            context_selector=lambda x: x.meta.get('col_normed_tfidf__n_feats',0)>= 5)

### Interpreting derived representations

Before applying the two transformers, `ec_fw` and `ec_bk` to transform the corpus, we can examine the representations and characterizations it's derived over the training data (note that in this case, the training data is also the corpus that we analyze, but this needn't be the case in general---see [this demo](https://github.com/CornellNLP/ConvoKit/blob/master/convokit/expected_context_framework/demos/wiki_awry_demo.ipynb) for an example).

First, to interpret the representations derived by each model, we can inspect the clusters of representations that we've inferred, for both the forwards and backwards direction. We can access the forwards and backwards models as elements of the `ec_models` attribute. The following function calls print out representative terms and utterances, as well as context terms and utterances, per cluster (next two cells; note that the output is quite long). 

In [16]:
dual_context_model.ec_models[0].print_clusters(corpus=sw_corpus)

CLUSTER 0 0
---
terms
        cluster_dist
index               
a           0.206770
i           0.236567
well        0.256482
and         0.259060
so          0.260796
it          0.298410
to          0.300810
really      0.311325
you         0.322042
uh          0.335718

context terms
        cluster_dist
index               
huh         0.391466
yeah        0.437190
i           0.450316
you         0.489002
oh          0.560207
that        0.561080
well        0.575842
it          0.608198
sounds      0.639354
okay        0.653139


utterances
> 3405-0 0.109 Okay , Lowell , so I 'd like to know , um , what , what do you do in lawn and garden , what , uh , what 's , what 's of interest to you and how do you go about it ?
> 2709-79 0.111 So , and I really do wish I knew about quotas and really wish I -
> 3014-69 0.114 So , I just trusted the CONSUMER REPORTS and the auto ,
> 4378-11 0.116 Well , I , I just live in a , I live in an apartment now . I , uh , two summers ago I went to Ma

In [17]:
dual_context_model.ec_models[1].print_clusters(corpus=sw_corpus)

CLUSTER 0 0
---
terms
         cluster_dist
index                
to           0.339085
that         0.340681
is           0.398213
not          0.407413
know         0.413964
for          0.421002
with         0.429641
about        0.429842
because      0.444867
me           0.447905

context terms
       cluster_dist
index              
that       0.464557
true       0.517588
do         0.558011
think      0.579059
know       0.750303
sure       0.751195
what       0.762769
no         0.825287
right      0.846140
how        0.870710


utterances
> 2303-21 0.109 Yeah , that 's , that 's possible . I still think that a lot of those people are the ones who really think that their votes do n't make a difference , though , as well . I think it 's those same people who do n't know any better about how we vote , are , are , are a lot of the people who think that well , look at me , I 'm just a little nobody . My vote 's not going to count anyway . You know , and I think that 's probably a p

>> 3728-80 0.217 Well , yeah , I do n't , I do n't , uh , -
>> 2041-112 0.230 Uh - huh . Yeah , well I ,
>> 2950-125 0.251 Well , I , that , that is really healthier , frankly .
>> 2956-80 0.259 # Well , it , uh , - well , I do n't know . #
>> 2744-162 0.262 # Uh - huh , well that 's , that 's it you , you do n't have , uh , #
>> 3971-33 0.266 Yeah , right . Well , I prefer ba- , uh , football --
>> 2181-107 0.266 You 're right , uh - huh , yeah , it 's true .
>> 3035-12 0.268 Uh - huh . Oh , yeah . I , -
>> 2019-25 0.271 Well that 's understandable , yeah , it 's , uh , -
>> 2107-35 0.280 Yes , uh - huh , that 's it

====



demo continues below

We can see that in each case, two clusters emerge that roughly correspond to utterances recounting personal experiences, and those providing commentary, generally not about personal matters. We'll label them as such, noting that there's a roughly 50-50 split with slightly more "personal" utterances than "commentary" ones:

In [18]:
dual_context_model.ec_models[0].set_cluster_names(['personal', 'commentary'])
dual_context_model.ec_models[1].set_cluster_names(['commentary', 'personal'])

### Interpreting derived characterizations

The transformer also computes some term-level statistics, which we can return as a Pandas dataframe:
* forwards and backwards ranges (`fw_range` and `bk_range` respectively): we roughly interpret these as modeling the strengths of our forwards expectations of the replies that a term tends to get, or the backwards expectations of the predecessors that the term tends to follow. 
* shift: this statistic corresponds to the distance between the backwards and forwards representations for each term; we interpret it as the extent to which a term shifts the focus of a conversation. 
* orientation (`orn`): this statistic compares the relative magnitude of forwards and backwards ranges. In a [counseling conversation setting](https://www.cs.cornell.edu/~cristian/Orientation_files/orientation-forwards-backwards.pdf) we interpreted orientation as a measure of the relative extent to which an interlocutor aims to advance the conversation forwards with a term, versus address existing content.

In [19]:
term_df = dual_context_model.get_term_df()

In [20]:
term_df.head()

orn     shift  bk_range  fw_range
index                                           
a        -0.009099  0.297876  0.804247  0.813346
able      0.017432  0.829677  0.831752  0.814321
about    -0.022503  0.380137  0.800652  0.823155
actually -0.006019  0.597292  0.790219  0.796238
after     0.029120  0.868623  0.835141  0.806022

In [21]:
k=10
print('low orientation')
display(term_df.sort_values('orn').head(k)[['orn']])
print('high orientation')
display(term_df.sort_values('orn').tail(k)[['orn']])
print('\nlow shift')
display(term_df.sort_values('shift').head(k)[['shift']])
print('high shift')
display(term_df.sort_values('shift').tail(k)[['shift']])

low orientation


orn
index              
how       -0.056177
done      -0.055084
let       -0.050308
few       -0.049332
basically -0.047425
nothing   -0.045719
nine      -0.043703
okay      -0.043319
whole     -0.042495
works     -0.042461

high orientation


orn
index            
taking   0.036944
makes    0.037245
change   0.038951
funny    0.039011
next     0.039310
called   0.043352
least    0.051271
anymore  0.051410
seem     0.064159
myself   0.064653


low shift


shift
index          
he     0.162183
to     0.173862
was    0.176154
car    0.195208
read   0.208384
him    0.214645
we     0.216772
for    0.218534
kids   0.223190
watch  0.223844

high shift


shift
index            
goes     1.104640
how      1.107404
exactly  1.117140
least    1.142747
change   1.162627
taking   1.206768
before   1.223504
nothing  1.270712
without  1.271164
tell     1.361794

### Deriving utterance-level representations

We now use the transformer to derive utterance-level characterizations, by transforming the corpus with it. Again, we focus on utterances that are sufficiently long:

In [22]:
_ = dual_context_model.transform(sw_corpus, selector=lambda x: x.meta.get('col_normed_tfidf__n_feats',0)>=5)

The `transform` function does the following. 

First, it (or rather, its constituent `ExpectedContextModelTransformer`s) derives vector representations of utterances, stored as `fw_repr` and `bk_repr`:

In [23]:
sw_corpus.vectors

{'bk_repr', 'col_normed_tfidf', 'fw_repr'}

Next, it derives ranges of utterances, stored in the metadata as `fw_range` and `bk_range`:

In [24]:
eg_ut = sw_corpus.get_utterance(utt_eg_id)
print('Forwards range:', eg_ut.meta['fw_range'])
print('Backwards range:', eg_ut.meta['bk_range'])

Forwards range: 0.8370251853798225
Backwards range: 0.8160712383960356


It also assigns utterances to inferred types:

In [25]:
print('Forwards cluster:', eg_ut.meta['fw_clustering.cluster'])
print('Backwards cluster:', eg_ut.meta['bk_clustering.cluster'])

Forwards cluster: personal
Backwards cluster: personal


And computes orientations and shifts:

In [26]:
print('shift:', eg_ut.meta['shift'])
print('orientation:', eg_ut.meta['orn'])

shift: 0.6741247781441236
orientation: -0.020953946983786942


## 3. Analysis: correspondence to discourse act labels

We explore the relation between the characterizations we've derived, and the various annotations that the utterances are labeled with (for more information on the annotation scheme, see the [manual here](https://web.stanford.edu/~jurafsky/ws97/manual.august1.html)). See [this dissertation](https://tisjune.github.io/research/dissertation) for further explanation of the analyses and findings below. A high-level comment is that this is a tough dataset for the framework to work with, given the relative lack of structure---something future work could think more carefully about.

To facilitate the analysis, we extract relevant utterance attributes into a Pandas dataframe:

In [27]:
df = sw_corpus.get_attribute_table('utterance',
                ['bk_clustering.cluster', 'fw_clustering.cluster',
                'orn', 'shift', 'tags'])
df = df[df['bk_clustering.cluster'].notnull()]

We will stick to examining the 9 most common tags in the data:

In [28]:
tag_subset = ['aa', 'b', 'ba', 'h', 'ny', 'qw', 'qy', 'sd', 'sv'] 
for tag in tag_subset:
    df['has_' + tag] = df.tags.apply(lambda x: tag in x.split())

To start, we explore how the forwards and backwards vector representations correspond to these labels. To do this, we will compute log-odds ratios between the inferred utterance clusters and these labels:

In [29]:
def compute_log_odds(col, bool_col, val_subset=None):
    if val_subset is not None:
        col_vals = val_subset
    else:
        col_vals = col.unique()
    log_odds_entries = []
    for val in col_vals:
        val_true = sum((col == val) & bool_col)
        val_false = sum((col == val) & ~bool_col)
        nval_true = sum((col != val) & bool_col)
        nval_false = sum((col != val) & ~bool_col)
        log_odds_entries.append({'val': val, 'log_odds': np.log((val_true/val_false)/(nval_true/nval_false))})
    return log_odds_entries

In [30]:
bk_log_odds = []
for tag in tag_subset:
    entry = compute_log_odds(df['bk_clustering.cluster'],df['has_' + tag], ['commentary'])[0]
    entry['tag'] = tag
    bk_log_odds.append(entry)
bk_log_odds_df = pd.DataFrame(bk_log_odds).set_index('tag').sort_values('log_odds')[['log_odds']]

In [31]:
fw_log_odds = []
for tag in tag_subset:
    entry = compute_log_odds(df['fw_clustering.cluster'],df['has_' + tag], ['commentary'])[0]
    entry['tag'] = tag
    fw_log_odds.append(entry)
fw_log_odds_df = pd.DataFrame(fw_log_odds).set_index('tag').sort_values('log_odds')[['log_odds']]

In [32]:
print('forwards types vs labels')
display(fw_log_odds_df.T)
print('--------------------------')
print('backwards types vs labels')
display(bk_log_odds_df.T)

forwards types vs labels


tag             qy        ny        qw        sd        ba         b  \
log_odds -0.491938 -0.485242 -0.472657 -0.380206 -0.304694 -0.159506   

tag             aa         h        sv  
log_odds  0.500705  0.718654  1.242259

--------------------------
backwards types vs labels


tag            ny        qy        sd        ba       qw         b       aa  \
log_odds -0.56057 -0.428184 -0.416996 -0.344168 -0.32995 -0.142323  0.50554   

tag              h        sv  
log_odds  0.733656  1.249692

Tags further towards the right of the above tables (more positive log-odds) are those that co-occur more with the `commentary` than the `personal` utterance type. We briefly note that both forwards and backwards representations seem to draw a distinction between `sv` (opinion statements) and `sd` (non-opinion statements).

Next, we explore how the orientation and shift statistics relate to these labels. To do this, we compare statistics for utterances with a particular label, to statistics for utterances without that label.

In [33]:
from scipy import stats

In [34]:
def cohend(d1, d2):
    n1, n2 = len(d1), len(d2)
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    u1, u2 = np.mean(d1), np.mean(d2)
    return (u1 - u2) / s
def get_pstars(p):
    if p  < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else: return ''

In [35]:
stat_col = 'orn'
entries = []
for tag in tag_subset:
    has = df[df['has_' + tag]][stat_col]
    hasnt = df[~df['has_' + tag]][stat_col]
    entry = {'tag': tag, 'pval': stats.mannwhitneyu(has, hasnt)[1],
            'cd': cohend(has, hasnt)}
    entry['ps'] = get_pstars(entry['pval'] * len(tag_subset))
    entries.append(entry)
orn_stat_df = pd.DataFrame(entries).set_index('tag').sort_values('cd')
orn_stat_df = orn_stat_df[np.abs(orn_stat_df.cd) >= .1]

In [36]:
stat_col = 'shift'
entries = []
for tag in tag_subset:
    has = df[df['has_' + tag]][stat_col]
    hasnt = df[~df['has_' + tag]][stat_col]
    entry = {'tag': tag, 'pval': stats.mannwhitneyu(has, hasnt)[1],
            'cd': cohend(has, hasnt)}
    entry['ps'] = get_pstars(entry['pval'] * len(tag_subset))
    entries.append(entry)
shift_stat_df = pd.DataFrame(entries).set_index('tag').sort_values('cd')
shift_stat_df = shift_stat_df[np.abs(shift_stat_df.cd) >= .1]

(We'll only show labels for which there's a sufficiently large difference, in cohen's delta, between utterances with and without the label)

In [37]:
print('orientation vs labels')
display(orn_stat_df.T)
print('--------------------------')
print('shift vs labels')
display(shift_stat_df.T)

orientation vs labels


tag            qw           sv           ba
cd      -0.464487     0.137032     0.325546
ps            ***          ***          ***
pval  4.57282e-58  3.04165e-41  1.04554e-35

--------------------------
shift vs labels


tag         sd            h           sv           ny           qy  \
cd   -0.514413    -0.457883    -0.245104    -0.162611     0.199682   
ps         ***          ***          ***          ***          ***   
pval         0  4.83333e-52  6.94752e-99  1.35491e-11  6.66404e-36   

tag            ba          qw  
cd       0.239983    0.493643  
ps            ***         ***  
pval  5.29252e-12  3.5594e-71

We note that utterances containing questions (`qw`, `qy`) have higher shifts than utterances which do not. If you're familiar with the DAMSL designations for forwards and backwards looking communicative functions, the output for orientation might look a little puzzling/informative that our view of what counts as forwards/backwards is different from the view espoused by the annotation scheme. We discuss this further in [this dissertation](https://tisjune.github.io/research/dissertation).

## 4. Model persistence

Finally, we briefly demonstrate how the model can be saved and loaded for later use

In [37]:
DUAL_MODEL_PATH = os.path.join(SW_CORPUS_PATH, 'dual_model')

In [39]:
dual_context_model.dump(DUAL_MODEL_PATH)

We dump latent context representations, clustering information, and various input parameters, for each constituent `ExpectedContextModelTransformer`, in separate directories under `DUAL_MODEL_PATH`:

In [40]:
ls $DUAL_MODEL_PATH

bk/  fw/


To load the learned model, we start by initializing a new model:

In [41]:
dual_model_new = DualContextWrapper(context_fields=['reply_to','next_id'], output_prefixes=['bk_new','fw_new'],
                                    vect_field='col_normed_tfidf', context_vect_field='col_normed_tfidf', 
                                    wrapper_output_prefix='new',
                                      n_svd_dims=15, n_clusters=2,
                                     random_state=1000, cluster_random_state=1000)

In [42]:
dual_model_new.load(DUAL_MODEL_PATH, model_dirs=['bk','fw'])

We see that using the re-loaded model to transform the corpus results in the same representations and characterizations as the original one:

In [43]:
_ = dual_model_new.transform(sw_corpus, selector=lambda x: x.meta.get('col_normed_tfidf__n_feats',0)>=5)

In [44]:
sw_corpus.vectors

{'bk_new_repr', 'bk_repr', 'col_normed_tfidf', 'fw_new_repr', 'fw_repr'}

In [45]:
np.allclose(sw_corpus.get_vectors('bk_new_repr'), sw_corpus.get_vectors('bk_repr'))

True

In [46]:
np.allclose(sw_corpus.get_vectors('fw_new_repr'), sw_corpus.get_vectors('fw_repr'))

True

In [47]:
for ut in sw_corpus.iter_utterances(selector=lambda x: x.meta.get('col_normed_tfidf__n_feats',0)>=5):
    assert ut.meta['orn'] == ut.meta['new_orn']
    assert ut.meta['shift'] == ut.meta['new_shift']

## 5. Pipeline usage

We also implement a pipeline that handles the following:
* processes text (via a pipeline supplied by the user)
* transforms text to input representation (via `ColNormedTfidfTransformer`)
* derives framework output (via `DualContextWrapper`)

In [38]:
from convokit.expected_context_framework import DualContextPipeline

In [39]:
# see `demo_text_pipelines.py` in this demo's directory for details
# in short, this pipeline will either output the `alpha_text` metadata  field
# of an utterance, or write the utterance's `text` attribute into the `alpha_text` 
# metadata field
from demo_text_pipelines import switchboard_text_pipeline

We initialize the pipeline with the following arguments:
* `text_field` specifies which utterance metadata field to use as text input
* `text_pipe` specifies the pipeline used to compute the contents of `text_field`
* `tfidf_params` specifies the parameters to be passed into the underlying `ColNormedTfidfTransformer` object
* `min_terms` specifies the minimum number of terms in the vocabulary that an utterance must contain for it to be considered in fitting and transforming the underlying `DualContextWrapper` object (see the `selector` argument passed into `dual_context_model.fit` above)

All other arguments are inherited from `DualContextWrapper`.

In [45]:
pipe_obj = DualContextPipeline(context_fields=['reply_to','next_id'], 
                output_prefixes=['bk','fw'],
                 text_field='alpha_text', text_pipe=switchboard_text_pipeline(),              
                tfidf_params={'binary': True, 'vocabulary': vocab}, 
                min_terms=5,
                n_svd_dims=15, n_clusters=2,
                 random_state=1000, cluster_random_state=1000)

In [46]:
# note this might output a warning that `col_normed_tfidf` already exists;
# that's okay: the pipeline is just recomputing this matrix
pipe_obj.fit(sw_corpus)

Note that the pipeline enables us to transform ad-hoc string input: 

In [47]:
eg_ut_new = pipe_obj.transform_utterance('How old were you when you left ?')

In [48]:
# note these attributes have the exact same values as those of eg_ut, computed above
print('shift:', eg_ut_new.meta['shift'])
print('orientation:', eg_ut_new.meta['orn'])

Forwards range: 0.8370251853798226
Backwards range: 0.8160712383960359
shift: 0.6741247781441236
orientation: -0.02095394698378672
